# Pymaceuticals Inc.
---

### Analysis
##### Analysis can go here or at the bottom of the file.  I usually put it at the end.
1.  
2.
3. 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single dataset
master_df = pd.merge(mouse_metadata, study_results, how='outer', on="Mouse ID")
# Display the data table for preview
master_df.head()

In [ ]:
# Checking the number of mice.
master_df['Mouse ID'].nunique()

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
clean_df = master_df.drop_duplicates(subset=['Mouse ID'])
clean_df.head()
duplicate_mice_IDs = clean_df.loc[master_df.duplicated(subset=['Mouse ID','Timepoint']),'Mouse ID'].unique()
duplicate_mice_IDs

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
duplicate_mice_df = master_df.loc[master_df['Mouse ID']=='g989', :]
duplicate_mice_df

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID (aka remove all records for that mouse)
# There are multiple ways of doing this and it is good to search online to see the suggestions.


In [ ]:
# Checking the number of mice in the clean DataFrame.
clean_df['Mouse ID'].nunique()
clean_mice_df = clean_df.loc[master_df['Mouse ID']=='g989', :]
clean_mice_df

## Summary Statistics

In [ ]:
# Method 1:  Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# by using groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

mean = clean_df['Tumor Volume (mm3)'].groupby(clean_df['Drug Regimen']).mean()
median = clean_df['Tumor Volume (mm3)'].groupby(clean_df['Drug Regimen']).median()
variance = clean_df['Tumor Volume (mm3)'].groupby(clean_df['Drug Regimen']).var()
std = clean_df['Tumor Volume (mm3)'].groupby(clean_df['Drug Regimen']).std()
sem = clean_df['Tumor Volume (mm3)'].groupby(clean_df['Drug Regimen']).sem()



summary_statistics = pd.DataFrame({'Mean Tumor Volume':mean,
                                   'Median Tumor Volume':median,
                                   'Tumor Volume Variance':variance,
                                   ' Standard Deviation of Tumor Volume':std,
                                   'SEM of Tumor Volume':sem,
                                  })

summary_statistics




In [ ]:
# Method 2:  Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# by using the aggregation method, produce the same summary statistics in a single line



## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using Pandas.


    mice_drug_regimen = master_df.groupby(["Drug Regimen"]).count()["Mouse ID"] 


plot_pandas = mice_drug_regimen.plot.bar(figsize=(15,10), color='b',fontsize = 14)
mice_drug_regimen
plt.xlabel("Drug Regimen",fontsize = 14)
plt.ylabel("Number of Mice",fontsize = 14)
plt.title("Number of Mice per Treatment",fontsize = 20)

# plt.savefig("../Images/mice_drug_regimen.png", bbox_inches = "tight")
plt.tight_layout()
plt.show()

mice_drug_regimen





In [ ]:
clean_df.tail()

In [ ]:
# Generate a bar plot showing the total number of timepoints for all mice tested for each drug regimen using pyplot.
mice_timepoint = clean_df["Timepoint"].value_counts()
mice_timepoint

x_axis = mice_timepoint.index.values
y_axis = mice_timepoint.values

# Create a Pyplot bar plot based off of the group series from before and label the title
plt.bar(x_axis, y_axis, color='b', alpha=0.8, align='center')

# Set the xlabel and ylabel, title using class methods
plt.title("Total Number of Timepoints for Mice Tested")
plt.xlabel("Timepoints")
plt.ylabel("Number of Mice")
plt.xticks(rotation="vertical")

plt.show()





In [ ]:
# Method1:  Generate a pie plot showing the distribution of female versus male mice using Pandas
# To clarify:  This answers the question:  of all the measurements taken, what percent where on male and female mice.
# This is different than:  of all the mice, what percent were male and female?

gender_df = clean_df["Sex"].value_counts()
plt.title("Female vs. Male Mice")
gender_df.plot.pie(autopct= "%1.1f%%")
plt.show()





In [ ]:
#Getting values for both Male and Female
gender_sum = gender_df['Male'] + gender_df['Female']
gender_sum
gender_male = gender_df['Male'] / gender_sum
gender_male
gender_female = gender_df['Female'] / gender_sum
gender_female

In [ ]:
# Method 2:  Generate a pie plot showing the distribution of female versus male mice using matplotlib (aka pyplot, plt)
# Same clarification as above.
labels = ['Female', 'Male']
sizes = [gender_female, gender_male]
plot = gender_df.plot.pie(y='Total Count', autopct="%1.1f%%")
plt.title('Male vs Female Mouse')
plt.ylabel('Sex')
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
# Start by getting the last (greatest) timepoint for each mouse
Ceftamin = clean_df.loc[clean_df["Drug Regimen"] == "Ceftamin", :]
Ramicane = clean_df.loc[clean_df["Drug Regimen"] == "Ramicane", :]
Capomulin = clean_df.loc[clean_df["Drug Regimen"] == "Capomulin",:]
Infubinol = clean_df.loc[clean_df["Drug Regimen"] == "Infubinol", :]

# Merge this group df with the original dataframe to get the tumor volume at the last timepoint
Capomulin_last = Capomulin.groupby('Mouse ID').max()['Timepoint']
Capomulin_vol = pd.DataFrame(Capomulin_last)
Capomulin_merge = pd.merge(Capomulin_vol, clean_df, on=("Mouse ID","Timepoint"),how="left")
Capomulin_merge.head()

In [ ]:
Infubinol.head()

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
# More information should be provided about this part since it is confusing and difficult.
# Part of this is similar to Day3 but **the suggestions make more sense** if you know you are doing
# this so so you have 4 lists contained inside a list bc this is used to generate the boxplot in the next cell.

treatments = ['Ceftamin','Ramicane','Capomulin','Infubinol']
# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    
    
    # add subset 
    
    
    # Determine outliers using upper and lower bounds

    
    
    
    
    
# Print results of each drugs outliers - results printed don't need to look pretty.  See below.    
# Note:  in the output below:  '31   36.32346' is series index 31 and value 36.32346.  There is only one outlier.



In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest





## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin









In [ ]:
# Generate a scatter plot of average tumor volume vs. average mouse weight of each mouse treated with the Capomulin regimen
# The chart below shows a dot for each mouse treated with Capomulin using average weight and average tumor Volume.
#  I would make the X-label say something like "Average Weight (g)"







## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen










